In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

In [48]:
print(torch.__version__)
print(torchvision.__version__)

1.3.0+cu100
0.4.1+cu100


In [0]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [0]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,12,5)

        self.fc1 = nn.Linear(12*4*4,120)
        self.fc2 = nn.Linear(120,60)
        self.out = nn.Linear(60,10)

    def forward(self,t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)

        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))

        t = F.relu(self.fc2(t))

        t = self.out(t)

        return t

In [0]:
train_set = torchvision.datasets.FashionMNIST(
    root = "./data",
    train=True,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor()]
    )
)

In [0]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)

Using tensorboard for network graph and images

In [0]:
tb = SummaryWriter()

network = Network()
images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images',grid)
tb.add_graph(network,images)
tb.close()

NOTE: storing the SummaryWriter outputs files on my local system to use a local host.<br>
saved on github as well.<br>
ngrok not working<br>


In [0]:
# use !zip -r <zipped file> <directory being zipped>

The training loop review

In [72]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)
optimizer = optim.Adam(network.parameters(),lr=0.01)

for epoch in range(1):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images,labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)

    print("epoch", epoch,'\n', "total_correct:",total_correct,'\n',"loss:",total_loss)


epoch 0 
 total_correct: 46327 
 loss: 362.071481525898


now including tensorboard calls in between to get data for analysis

In [0]:
!rm -r runs

In [74]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)
optimizer = optim.Adam(network.parameters(),lr=0.01)

images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images',grid)
tb.add_graph(network,images)

for epoch in range(1):
    total_loss = 0
    total_correct = 0

    for batch in train_loader: # getting batch
        images,labels = batch # separating images and labels
        
        preds = network(images) 
        loss = F.cross_entropy(preds,labels) #calculating loss

        optimizer.zero_grad() 
        loss.backward()  # calculating gradients
        optimizer.step() # updating weights

        tb.add_scalar('Loss',loss,epoch)
        tb.add_scalar('Number correct',total_correct,epoch)
        tb.add_scalar('Accuracy',total_correct/len(train_set),epoch)

        tb.add_histogram('conv1.bias',network.conv1.bias,epoch)
        tb.add_histogram('conv1.weight',network.conv1.weight,epoch)
        tb.add_histogram('conv1.weight.grad',network.conv1.weight.grad,epoch)

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)

    print("epoch", epoch,'\n', "total_correct:",total_correct,'\n',"loss:",total_loss)

tb.close()

epoch 0 
 total_correct: 47308 
 loss: 339.0733009278774


In [76]:
!zip -r runs_0 ./runs

  adding: runs/ (stored 0%)
  adding: runs/Oct28_16-50-13_9f396b488efb/ (stored 0%)
  adding: runs/Oct28_16-50-13_9f396b488efb/events.out.tfevents.1572281413.9f396b488efb.131.5 (deflated 97%)
